In [ ]:
using InteractiveUtils, Pkg, Test

Base.banner()

Pkg.activate(@__DIR__)
Pkg.status()

using JET

# Basic Usage

- Interactive entry point: `[@]report_call`, `[@]report_opt`
  - `report_call` for detecting bugs
  - `report_opt` for detecting performance problems
- Works like `[@]code_typed`

In [ ]:
report_call(sum, (Vector{Int},))

In [ ]:
@report_call sum("julia")

# Notable features for package development

- Test.jl integration
- `report_package`
- Various configurations
  * Typo detection mode
  * Report filters

# Test.jl integration

JET provides smooth integration with Test.jl:
- Allows users to use JET in their test suite
  - `[@]report_call`, `[@]report_opt` <-> `[@]test_call`, `[@]test_opt`
  - `report_package`, `report_file` <-> `test_package`, `test_file`
- Results are included in the test set summary
- Also supports Test.jl features like `broken` and `skip`

Packages using JET in CI include: JET, Graphs.jl, CliMA ecosystem

In [ ]:
function test_func(a)
    return a + 1
end

@test_call test_func("julia")

In [ ]:
@testset "JET code quality test" begin
    @test_call test_func("julia")
    @test_call test_func(1)
end

In [ ]:
@testset "JET code quality test" begin
    @test_call skip=true test_func("julia")
    @test_call test_func(1)
end

# `report_package`

- An easy analysis entry point at the cost of analysis accuracy
- Improved affinity with Pkg.jl system
  * No need to instantiate/activate a target package environment beforehand
- `target_defined_modules` configuration
  * Allows you to focus on your code

In [14]:
report_package("Demo1")

[toplevel-info] virtualized the context of Main (took 0.001 sec)
[toplevel-info] entered into /Users/aviatesk/julia/JET-20230703/Demo1/src/Demo1.jl
[toplevel-info]  exited from /Users/aviatesk/julia/JET-20230703/Demo1/src/Demo1.jl (took 0.002 sec)
[toplevel-info] 

[toplevel-info] analyzing from top-level definitions (3/3)
[toplevel-info] analyzed 3 top-level definitions (took 0.024 sec)


═════ 2 possible errors found ═════
┌ callsum3(a::String) @ Demo1 /Users/aviatesk/julia/JET-20230703/Demo1/src/Demo1.jl:7
│┌ sum(a::String) @ Base ./reduce.jl:559
││┌ sum(a::String; kw::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}) @ Base ./reduce.jl:559
│││┌ sum(f::typeof(identity), a::String) @ Base ./reduce.jl:530
││││┌ sum(f::typeof(identity), a::String; kw::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}) @ Base ./reduce.jl:530
│││││┌ mapreduce(f::typeof(identity), op::typeof(Base.add_sum), itr::String) @ Base ./reduce.jl:302
││││││┌ mapreduce(f::typeof(identity), op::typeof(Base.add_sum), itr::String; kw::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}) @ Base ./reduce.jl:302
│││││││┌ mapfoldl(f::typeof(identity), op::typeof(Base.add_sum), itr::String) @ Base ./reduce.jl:170
││││││││┌ mapfoldl(f::typeof(identity), op::typeof(Base.add_sum), itr::String; init::Base._InitialValue) @ Base ./reduce.jl:170
│││││││││┌ mapfoldl_impl(f::typeof(

In [15]:
report_package("AbstractTrees"; mode=:typo)

**WARNING**:

`report_package(YourPkg::Module; target_defined_modules=true)` does not work as expected.
Use `report_package("YourPkg"::String; target_defined_modules=true)` instead.